# Exercise 02: Backpropagation Made Visible

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shang-vikas/series1-coding-exercises/blob/main/exercises/blog-03/exercise-02.ipynb)

## Setup

In [4]:
# Install required packages using the kernel's Python interpreter
import sys
import subprocess
import importlib

def install_if_missing(package, import_name=None):
    """Install package if it's not already installed."""
    if import_name is None:
        import_name = package

    try:
        importlib.import_module(import_name)
        print(f"✓ {package} is already installed")
    except ImportError:
        print(f"Installing {package}....")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installed successfully")

# Install required packages
install_if_missing("numpy")

✓ numpy is already installed


## Tiny Network

We'll build the smallest possible 2-layer network and print the "blame" numbers flowing backward.

No frameworks. Just NumPy. No symbolic calculus. Just numbers.

**Structure:**

```
x → Linear1 → ReLU → Linear2 → output → loss
```

Single training example. One step.

### Forward + Backward (With Printed Blame)

In [5]:
import numpy as np

# ----- Data -----
x = np.array([1.0, 2.0])       # 2 input features
y_true = np.array([1.0])       # target

# ----- Parameters -----
W1 = np.array([[0.5, -0.3],
               [0.8,  0.2]])   # shape (2,2)
b1 = np.array([0.1, -0.1])

W2 = np.array([[1.0],
               [-1.5]])        # shape (2,1)
b2 = np.array([0.05])

lr = 0.1

# ----- Forward Pass -----
z1 = x @ W1 + b1
a1 = np.maximum(0, z1)   # ReLU
z2 = a1 @ W2 + b2
y_pred = z2              # no activation for simplicity

loss = np.mean((y_pred - y_true)**2)

print("Forward:")
print(" z1:", z1)
print(" a1:", a1)
print(" y_pred:", y_pred)
print(" loss:", loss)

# ----- Backward Pass -----

# dLoss/dy_pred
d_loss_y = 2 * (y_pred - y_true)

# Gradients for W2 and b2
dW2 = np.outer(a1, d_loss_y)
db2 = d_loss_y

# Propagate blame to a1
d_a1 = d_loss_y @ W2.T

# ReLU local gradient
d_z1 = d_a1 * (z1 > 0)

# Gradients for W1 and b1
dW1 = np.outer(x, d_z1)
db1 = d_z1

print("\nBackward (Blame Signals):")
print(" d_loss_y (blame at output):", d_loss_y)
print(" dW2 (blame for W2):\n", dW2)
print(" d_a1 (blame flowing to layer1 output):", d_a1)
print(" d_z1 (after ReLU gate):", d_z1)
print(" dW1 (blame for W1):\n", dW1)

# ----- Update -----
W1 -= lr * dW1
b1 -= lr * db1
W2 -= lr * dW2
b2 -= lr * db2

print("\nUpdated W1:\n", W1)
print("Updated W2:\n", W2)

Forward:
 z1: [2.20000000e+00 2.77555756e-17]
 a1: [2.20000000e+00 2.77555756e-17]
 y_pred: [2.25]
 loss: 1.5625

Backward (Blame Signals):
 d_loss_y (blame at output): [2.5]
 dW2 (blame for W2):
 [[5.5000000e+00]
 [6.9388939e-17]]
 d_a1 (blame flowing to layer1 output): [ 2.5  -3.75]
 d_z1 (after ReLU gate): [ 2.5  -3.75]
 dW1 (blame for W1):
 [[ 2.5  -3.75]
 [ 5.   -7.5 ]]

Updated W1:
 [[0.25  0.075]
 [0.3   0.95 ]]
Updated W2:
 [[ 0.45]
 [-1.5 ]]


## What Readers Should Notice

### `d_loss_y`
The first blame signal.
Just: "How wrong was the output?"

### `dW2`
Blame assigned to last layer weights.
Larger `a1` → larger blame on corresponding weight.

### `d_a1`
Blame flows backward through `W2`.
If `W2` is large, earlier layers inherit larger responsibility.

### `d_z1`
ReLU acts like a gate.
If a neuron was inactive, its blame is zero.
Dead neurons don't get blamed.

### `dW1`
Now earlier weights receive proportional blame.

**No neuron "realizes" anything.**

Numbers just flow backward based on local sensitivities.

## Engineer Interpretation

Think of each `dSomething` as:

**"If this value had been slightly different, how much would the final loss change?"**

That's it.

Backprop is just:

1. compute output error
2. propagate responsibility backward
3. update knobs proportionally

## Key Insight

**Backpropagation is not intelligence.**

It is a systematic way of distributing blame across chained computations.

## 🧠 Blame Flow Visual Demo

This prints gradient magnitudes as bars so readers see intensity.

In [6]:
import numpy as np

def bar(x):
    return "|" * int(abs(x) * 10)

# ----- Data -----
x = np.array([1.0, 2.0])
y_true = np.array([1.0])

# ----- Parameters -----
W1 = np.array([[0.5, -0.3],
               [0.8,  0.2]])
b1 = np.array([0.1, -0.1])

W2 = np.array([[1.0],
               [-1.5]])
b2 = np.array([0.05])

# ----- Forward -----
z1 = x @ W1 + b1
a1 = np.maximum(0, z1)
z2 = a1 @ W2 + b2
y_pred = z2

loss = np.mean((y_pred - y_true)**2)

print("\nFORWARD PASS")
print("Layer1 activations:", a1)
print("Output:", y_pred)
print("Loss:", loss)

# ----- Backward -----
d_loss_y = 2 * (y_pred - y_true)
dW2 = np.outer(a1, d_loss_y)
d_a1 = d_loss_y @ W2.T
d_z1 = d_a1 * (z1 > 0)
dW1 = np.outer(x, d_z1)

print("\nBACKWARD PASS (Blame Intensity)")
print("\nOutput blame:", d_loss_y, bar(d_loss_y[0]))

print("\nLayer2 weight blame:")
for i, val in enumerate(dW2.flatten()):
    print(f"W2[{i}] →", val, bar(val))

print("\nLayer1 activation blame:")
for i, val in enumerate(d_a1.flatten()):
    print(f"a1[{i}] →", val, bar(val))

print("\nLayer1 weight blame:")
for i, val in enumerate(dW1.flatten()):
    print(f"W1[{i}] →", val, bar(val))


FORWARD PASS
Layer1 activations: [2.20000000e+00 2.77555756e-17]
Output: [2.25]
Loss: 1.5625

BACKWARD PASS (Blame Intensity)

Output blame: [2.5] |||||||||||||||||||||||||

Layer2 weight blame:
W2[0] → 5.5 |||||||||||||||||||||||||||||||||||||||||||||||||||||||
W2[1] → 6.938893903907228e-17 

Layer1 activation blame:
a1[0] → 2.5 |||||||||||||||||||||||||
a1[1] → -3.75 |||||||||||||||||||||||||||||||||||||

Layer1 weight blame:
W1[0] → 2.5 |||||||||||||||||||||||||
W1[1] → -3.75 |||||||||||||||||||||||||||||||||||||
W1[2] → 5.0 ||||||||||||||||||||||||||||||||||||||||||||||||||
W1[3] → -7.5 |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||


### 🔍 What This Demonstrates Visually

When loss is high:

- Output layer shows strong blame.
- That blame distributes backward.
- Earlier layers get weaker signals.
- If a ReLU neuron was inactive → zero bars.

It becomes obvious:

**Backprop is just blame propagation through multiplications.**

## 🧠 Visual Diagram

**FORWARD:**
```
x → [Layer1] → [Layer2] → y_pred → loss
```

**BACKWARD:**
```
loss → blame → Layer2 → blame → Layer1
```